## Preparing our dataset

Over the past few years, streaming services with huge catalogs have become the primary means through which most peaple listen to teir favorite music, but at the same time, the sheer amoint of music on offer can mean users might be a bit overwhelmed when trying t olook for newer music their tastes.

for this reason, streaming serives have looked into means of categorizing music to allow for personalized recommendation, one method involves direct analysis of the raw audio information in a given song, scoring the raw data on a variety of matrics, today we will be examining data compiled by a research group known a the ECHO NEST, our goal is to look through this dataset and classify songs as being either 'hip hop' or 'rock' - all wothout listening to a single one ourselves, in doing so n we will learn how to clean some simple machine learning algorithms. such as decision trees and logistic regression.
 to begin with, lets load the metadata about our tracks alongside the track such as danceablity and acousticness on a scale from -1 to 1. these exist in two different files, which are in different in which databases often return the results of a given query.
 

 

In [2]:
## importation de library
import pandas as pd 

In [8]:
## Read in track metadata with genre labels :
url = (r'https://raw.githubusercontent.com/anuraglahon16/DataCamp-Projects-Solutions/master/Classify%20Song%20Genres%20from%20Audio%20Data/fma-rock-vs-hiphop.csv')
url_json = (r'https://raw.githubusercontent.com/anuraglahon16/DataCamp-Projects-Solutions/master/Classify%20Song%20Genres%20from%20Audio%20Data/echonest-metrics.json')

## Read in track metrics with the features :
tracks = pd.read_csv(url)
echonest_metrics = pd.read_json(url_json)



In [9]:
print(tracks.head(2))
print(echonest_metrics.head(2))


   track_id  bit_rate  comments  ... publisher tags                       title
0       135    256000         1  ...       NaN   []                Father's Day
1       136    256000         1  ...       NaN   []  Peel Back The Mountain Sky

[2 rows x 21 columns]
   track_id  acousticness  danceability  ...  speechiness    tempo   valence
0         2      0.416675      0.675894  ...     0.159310  165.922  0.576661
1         3      0.374408      0.528643  ...     0.461818  126.957  0.269240

[2 rows x 9 columns]


In [10]:
print(tracks.columns)
print(echonest_metrics.columns)

Index(['track_id', 'bit_rate', 'comments', 'composer', 'date_created',
       'date_recorded', 'duration', 'favorites', 'genre_top', 'genres',
       'genres_all', 'information', 'interest', 'language_code', 'license',
       'listens', 'lyricist', 'number', 'publisher', 'tags', 'title'],
      dtype='object')
Index(['track_id', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'speechiness', 'tempo', 'valence'],
      dtype='object')


In [5]:
## Merge the relevant columns of tracks and echonest_metrics
echo_tracks = pd.merge(left = echonest_metrics, right= tracks[['track_id','genre_top']], on = 'track_id')

## Inspect the resultant dataframe.
echo_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4802 entries, 0 to 4801
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          4802 non-null   int64  
 1   acousticness      4802 non-null   float64
 2   danceability      4802 non-null   float64
 3   energy            4802 non-null   float64
 4   instrumentalness  4802 non-null   float64
 5   liveness          4802 non-null   float64
 6   speechiness       4802 non-null   float64
 7   tempo             4802 non-null   float64
 8   valence           4802 non-null   float64
 9   genre_top         4802 non-null   object 
dtypes: float64(8), int64(1), object(1)
memory usage: 412.7+ KB


## Pairwise relationships between continuous variable:

we typically want to avoid using variables that have strong correlation with each other,

 -- hence avoidiing fearue redundancy for a few reasons--
 -- To keep the model simple and imporve interpretablity (with many featurs we run the risk of overfitting)
 -- When our datasets are very large, using fewer feature can drastically speed up our computation time.



In [13]:
# Create a correlation matrix :
corr_metrics = echo_tracks.corr()
corr_metrics.style.background_gradient()

,track_id,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,1.000000,-0.372282,0.049454,0.140703,-0.275623,0.048231,-0.026995,-0.025392,0.010070
acousticness,-0.372282,1.000000,-0.028954,-0.281619,0.194780,-0.019991,0.072204,-0.026310,-0.013841
danceability,0.049454,-0.028954,1.000000,-0.242032,-0.255217,-0.106584,0.276206,-0.242089,0.473165
energy,0.140703,-0.281619,-0.242032,1.000000,0.028238,0.113331,-0.109983,0.195227,0.038603
instrumentalness,-0.275623,0.194780,-0.255217,0.028238,1.000000,-0.091022,-0.366762,0.022215,-0.219967
liveness,0.048231,-0.019991,-0.106584,0.113331,-0.091022,1.000000,0.041173,0.002732,-0.045093
speechiness,-0.026995,0.072204,0.276206,-0.109983,-0.366762,0.041173,1.000000,0.008241,0.149894
tempo,-0.025392,-0.026310,-0.242089,0.195227,0.022215,0.002732,0.008241,1.000000,0.052221
valence,0.010070,-0.013841,0.473165,0.038603,-0.219967,-0.045093,0.149894,0.052221,1.000000


## Normalizing the feature data : 
As mentioned earlier, it can be particularly useful to simplify our models and use as few features as necessary to achieve the best result. Since we didn't find any particular strong correlations between our features, we can instead use a common approach to reduce the number of features called principal component analysis (PCA).

It is possible that the variance between genres can be explained by just a few features in the dataset. PCA rotates the data along the axis of highest variance, thus allowing us to determine the relative contribution of each feature of our data towards the variance between classes.

However, since PCA uses the absolute variance of a feature to rotate the data, a feature with a broader range of values will overpower and bias the algorithm relative to the other features. To avoid this, we must first normalize our data. There are a few methods to do this, but a common way is through standardization, such that all features have a mean = 0 and standard deviation = 1 (the resultant is a z-score).

In [14]:
# Define our features 
features = echo_tracks.drop(['genre_top', 'track_id'], axis = 1)

# Define our labels :
labels = echo_tracks['genre_top']

# import the StandardScaler 
from sklearn.preprocessing import StandardScaler

# Scale  the feature and set the values to a new variable 
scaler = StandardScaler()
scaler_train_features = scaler.fit_transform(features)


## Principal component analysis on our scaled data
Now that we have preprocessed our data, we are ready to use PCA to determine by how much we can reduce the dimensionality of our data, we can use scree-plots and cumulative explained ratio plots to find the number of components to use in further analysis.
scree-plots display the number of components against the variance explained by each component. sorted in descending order of variance. scree-plot help us get a better sense of which components explain a sufficient amount of variance in our data.
when using scree plots, an 'elbow' (a steep drop from one data point to the next) in the plot is typically used to decide on an appropriate cutoff.


In [ ]:
# This is just to make plots appear in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
## Get our explained variance ratios from PCA using all features 
pca = PCA()
pca.fit(scaler_train_features)
exp_variance = pca.explained